## Install Packages

In [1]:
# pyspark

!pip install pyspark

In [2]:
!pip install -U textblob

In [3]:
!pip install tensorflow

In [4]:
!pip install gensim

In [5]:
!pip install wordcloud

In [6]:
!pip install plotly

In [7]:
!pip install statsmodels

In [8]:
!pip install scikit-learn

In [9]:
!pip install statsmodels

## Import Libraries

In [10]:
################################################### spark #################################################################

from pyspark.sql import SparkSession

################################################### base #################################################################

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

################################################### nlp #################################################################

import numpy as np

import gensim
import nltk
import re
from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer

################################################### time series #################################################################



from statsmodels.tsa.seasonal import seasonal_decompose
from nltk.corpus import stopwords as nltk_stopwords

###################################################  ml #################################################################

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional,Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

################################################# confusion matrix #################################################################

from sklearn.metrics import confusion_matrix

## Create Spark

In [11]:
# Create a Spark session
spark = SparkSession.builder.appName('ProjectTweets').getOrCreate()

In [12]:
# spark

spark

## Viewing Data

In [16]:
# upload dataset
file_path = '/content/ProjectTweets.csv'  # Replace with the actual path to your dataset
tweets = spark.read.csv(file_path, header=True, inferSchema=True)

# Show the DataFrame
tweets.show(10)

+---+----------+----------------------------+--------+---------------+-------------------------------------------------------------------------------------------------------------------+
|  0|1467810369|Mon Apr 06 22:19:45 PDT 2009|NO_QUERY|_TheSpecialOne_|@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D|
+---+----------+----------------------------+--------+---------------+-------------------------------------------------------------------------------------------------------------------+
|  1|1467810672|        Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|                                                                                               is upset that he ...|
|  2|1467810917|        Mon Apr 06 22:19:...|NO_QUERY|       mattycus|                                                                                               @Kenichan I dived...|
|  3|1467811184|        Mon Apr 06 22:19:...|NO_QUERY|        Ell

In [14]:
tweets.show(5)

+---+----------+----------------------------+--------+---------------+-------------------------------------------------------------------------------------------------------------------+
|  0|1467810369|Mon Apr 06 22:19:45 PDT 2009|NO_QUERY|_TheSpecialOne_|@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D|
+---+----------+----------------------------+--------+---------------+-------------------------------------------------------------------------------------------------------------------+
|  1|1467810672|        Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|                                                                                               is upset that he ...|
|  2|1467810917|        Mon Apr 06 22:19:...|NO_QUERY|       mattycus|                                                                                               @Kenichan I dived...|
|  3|1467811184|        Mon Apr 06 22:19:...|NO_QUERY|        Ell

In [15]:
tweets.columns

['0',
 '1467810369',
 'Mon Apr 06 22:19:45 PDT 2009',
 'NO_QUERY',
 '_TheSpecialOne_',
 "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"]

In [16]:
tweets.printSchema()

root
 |-- 0: integer (nullable = true)
 |-- 1467810369: long (nullable = true)
 |-- Mon Apr 06 22:19:45 PDT 2009: string (nullable = true)
 |-- NO_QUERY: string (nullable = true)
 |-- _TheSpecialOne_: string (nullable = true)
 |-- @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D: string (nullable = true)



In [17]:
tweets.collect()

[Row(0=1, 1467810369=1467810672, Mon Apr 06 22:19:45 PDT 2009='Mon Apr 06 22:19:49 PDT 2009', NO_QUERY='NO_QUERY', _TheSpecialOne_='scotthamilton', @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D="is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!"),
 Row(0=2, 1467810369=1467810917, Mon Apr 06 22:19:45 PDT 2009='Mon Apr 06 22:19:53 PDT 2009', NO_QUERY='NO_QUERY', _TheSpecialOne_='mattycus', @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D='@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds'),
 Row(0=3, 1467810369=1467811184, Mon Apr 06 22:19:45 PDT 2009='Mon Apr 06 22:19:57 PDT 2009', NO_QUERY='NO_QUERY', _TheSpecialOne_='ElleCTF', @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D='my whole

In [18]:
tweets.take(5)

[Row(0=1, 1467810369=1467810672, Mon Apr 06 22:19:45 PDT 2009='Mon Apr 06 22:19:49 PDT 2009', NO_QUERY='NO_QUERY', _TheSpecialOne_='scotthamilton', @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D="is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!"),
 Row(0=2, 1467810369=1467810917, Mon Apr 06 22:19:45 PDT 2009='Mon Apr 06 22:19:53 PDT 2009', NO_QUERY='NO_QUERY', _TheSpecialOne_='mattycus', @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D='@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds'),
 Row(0=3, 1467810369=1467811184, Mon Apr 06 22:19:45 PDT 2009='Mon Apr 06 22:19:57 PDT 2009', NO_QUERY='NO_QUERY', _TheSpecialOne_='ElleCTF', @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D='my whole

## Add column names and convert to pandas

In [37]:
tweets = tweets.toPandas()
tweets.columns = ['target', 'id', 'date', 'query', 'username', 'content']
tweets.head()

,target,id,date,query,username,content
0,1,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,2,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,3,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,4,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,5,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


## Dtypes

In [39]:
tweets.dtypes

target       int32
id           int64
date        object
query       object
username    object
content     object
dtype: object

## Cheking Missing Values

In [40]:
tweets.isna().sum()

target      0
id          0
date        0
query       0
username    0
content     0
dtype: int64